**Introduction**

A T5 is an encoder-decoder model. It converts all NLP problems like language translation, summarization, text generation, question-answering, to a text-to-text task.A T5 is an encoder-decoder model. It converts all NLP problems like language translation, summarization, text generation, question-answering, to a text-to-text task.

For e.g., in case of **translation**, T5 accepts source text: **English**, as input and tries to convert it into target text: **Serbian**

In case of **summarization**, source text or input can be a long description and target text can just be a one line summary.

In this article, we will take a pretrained **T5-base** model and fine tune it to generate a one line summary of news articles using **PyTorch**.

**Data**

We will take a news summary dataset: It has 2 columns:

* **text** : article content
* **headlines** : one line summary of article content

In [1]:
import pandas as pd

path = "/kaggle/input/full-data/dataset.csv"
df = pd.read_csv(path)

df.head()

,File No.,Judgment,Summary
0,1,Appeal No. LXVI of 1949.\nAppeal from the High...,The charge created in respect of municipal pro...
1,10,Civil Appeal No.94 of 1949.\n107 834 Appeal fr...,"An agreement for a lease, which a lease is by ..."
2,100,"iminal Appeal No. 40 of 1951, 127 Appeal from ...","The question whether a Magistrate is ""personal..."
3,1000,Appeal No. 388 of 1960.\nAppeal by special lea...,The appellant was a member of a joint Hindu fa...
4,1001,Appeal No. 198 of 1954.\nAppeal from the judgm...,The appellant was the Ruler of the State of Ba...


**PyTorch** has a standard way to train any deep learning model. We will first start by writing a **Dataset** class, followed by **training**, **validation** steps and then a main **T5Trainer** function that will fine-tune our model.

**Install Libraries**

In [2]:
#!pip install sentencepiece
#!pip install transformers
#!pip install torch
#!pip install rich[jupyter]

**Import Libraries**

In [3]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

# rich: for a better display on terminal
from rich.table import Column, Table
from rich import box
from rich.console import Console

In [4]:
# define a rich console logger
console = Console(record=True)

# to display dataframe in ASCII format
def display_df(df):
    """display dataframe in ASCII format"""

    console = Console()
    table = Table(
        Column("source_text", justify="center"),
        Column("target_text", justify="center"),
        title="Sample Data",
        pad_edge=False,
        box=box.ASCII,
    )

    for i, row in enumerate(df.values.tolist()):
        table.add_row(row[0], row[1])

    console.print(table)

# training logger to log training progress
training_logger = Table(
    Column("Epoch", justify="center"),
    Column("Steps", justify="center"),
    Column("Loss", justify="center"),
    title="Training Status",
    pad_edge=False,
    box=box.ASCII,
)

In [5]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

**Dataset class**

Dataset class is for reading the dataset and loading it into the dataloader and then feed it to the transformer for fine tuning the model.

This class will take 6 arguments as input:

* **dataframe (pandas.DataFrame)**: Input dataframe
* **tokenizer (transformers.tokenizer)**: T5 tokenizer
* **source_len (int)**: Max length of source text
* **target_len (int)**: Max length of target text
* **source_text (str)**: column name of source text
* **target_text (str)**: column name of target text

This class will have 2 methods:

* **__len__**: returns the length of the dataframe
* **__getitem__**: return the input ids, attention masks and target ids

In [6]:
class CustomDataSetClass(Dataset):
    """
    Creating a custom dataset for reading the dataset and
    loading it into the dataloader to pass it to the
    transformer for finetuning the model
    """
    
    def __init__(
        self, dataframe, tokenizer, source_len, target_len, source_text, target_text
    ):
        """
        Initializes a Dataset class

        Args:
            dataframe (pandas.DataFrame): Input dataframe
            tokenizer (transformers.tokenizer): Transformers tokenizer
            source_len (int): Max length of source text
            target_len (int): Max length of target text
            source_text (str): column name of source text
            target_text (str): column name of target text
        """
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = target_len
        self.target_text = self.data[target_text]
        self.source_text = self.data[source_text]

    def __len__(self):
        """returns the length of dataframe"""

        return len(self.target_text)

    def __getitem__(self, index):
        """return the input ids, attention masks and target ids"""

        source_text = str(self.source_text[index])
        target_text = str(self.target_text[index])

        # cleaning data so as to ensure data is in string type
        source_text = " ".join(source_text.split())
        target_text = " ".join(target_text.split())

        source = self.tokenizer.batch_encode_plus(
            [source_text],
            max_length=self.source_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        target = self.tokenizer.batch_encode_plus(
            [target_text],
            max_length=self.summ_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()
        target_ids = target["input_ids"].squeeze()
        target_mask = target["attention_mask"].squeeze()

        return {
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
            "target_ids": target_ids.to(dtype=torch.long),
            "target_ids_y": target_ids.to(dtype=torch.long),
        }

**Model Training**

**train** function will the put model on training mode, generate outputs and calculate loss

This will take 6 arguments as input:

* **epoch**: epoch
* **tokenizer**: T5 tokenizer
* **model**: T5 model
* **loader**: Train Dataloader
* **optimizer**: Optimizer

In [7]:
def train(epoch, tokenizer, model, device, loader, optimizer):

    """
    Function to be called for training with the parameters passed from main function

    """

    model.train()
    for _, data in enumerate(loader, 0):
        y = data["target_ids"].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data["source_ids"].to(device, dtype=torch.long)
        mask = data["source_mask"].to(device, dtype=torch.long)

        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            decoder_input_ids=y_ids,
            labels=lm_labels,
        )
        loss = outputs[0]

        if _ % 100 == 0:
            training_logger.add_row(str(epoch), str(_), str(loss))
            console.print(training_logger)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

**Validation**

validate function is same as the train function, but for the validation data


In [8]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals

**T5 Trainer**

**T5Trainer** is our main function. It accepts input data, model type, model paramters to fine-tune the model. Under the hood, it utilizes, our Dataset class for data handling, train function to fine tune the model, validate to evaluate the model.

T5Trainer will have 5 arguments:

* **dataframe**: Input dataframe
* **source_text**: Column name of the input text i.e. article content
* **target_text**: Column name of the taregt text i.e. one line summary
* **model_params**: T5 model parameters
* **output_dir**: Output directory to save fine tuned T5 model.

In [9]:
def T5Trainer(
    dataframe, source_text, target_text, model_params, output_dir="/kaggle/working/"
):

    """
    T5 trainer

    """

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(model_params["SEED"])  # pytorch random seed
    np.random.seed(model_params["SEED"])  # numpy random seed
    torch.backends.cudnn.deterministic = True

    # logging
    console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary.
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
    model = model.to(device)

    # logging
    console.log(f"[Data]: Reading data...\n")

    # Importing the raw dataset
    dataframe = dataframe[[source_text, target_text]]
    display_df(dataframe.head(2))

    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest for validation.
    train_size = 0.8
    train_dataset = dataframe.sample(frac=train_size, random_state=model_params["SEED"])
    val_dataset = dataframe.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    console.print(f"FULL Dataset: {dataframe.shape}")
    console.print(f"TRAIN Dataset: {train_dataset.shape}")
    console.print(f"TEST Dataset: {val_dataset.shape}\n")

    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = CustomDataSetClass(
        train_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )
    val_set = CustomDataSetClass(
        val_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )

    # Defining the parameters for creation of dataloaders
    train_params = {
        "batch_size": model_params["TRAIN_BATCH_SIZE"],
        "shuffle": True,
        "num_workers": 0,
    }

    val_params = {
        "batch_size": model_params["VALID_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0,
    }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)

    # Defining the optimizer that will be used to tune the weights of the network in the training session.
    optimizer = torch.optim.Adam(
        params=model.parameters(), lr=model_params["LEARNING_RATE"]
    )

    # Training loop
    console.log(f"[Initiating Fine Tuning]...\n")

    for epoch in range(model_params["TRAIN_EPOCHS"]):
        train(epoch, tokenizer, model, device, training_loader, optimizer)

    console.log(f"[Saving Model]...\n")
    # Saving the model after training
    path = os.path.join(output_dir, "model_files")
    model.save_pretrained(path)
    tokenizer.save_pretrained(path)

    # evaluating test dataset
    console.log(f"[Initiating Validation]...\n")
    for epoch in range(model_params["VAL_EPOCHS"]):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({"Generated Text": predictions, "Actual Text": actuals})
        final_df.to_csv(os.path.join(output_dir, "predictions.csv"))

    console.save_text(os.path.join(output_dir, "logs.txt"))

    console.log(f"[Validation Completed.]\n")
    console.print(
        f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n"""
    )
    console.print(
        f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n"""
    )
    console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")
    
    return final_df

**Model Parameters**

In [10]:

# let's define model parameters specific to T5
model_params = {
    "MODEL": "t5-base",  # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE": 8,  # training batch size
    "VALID_BATCH_SIZE": 8,  # validation batch size
    "TRAIN_EPOCHS": 1,  # number of training epochs
    "VAL_EPOCHS": 1,  # number of validation epochs
    "LEARNING_RATE": 1e-4,  # learning rate
    "MAX_SOURCE_TEXT_LENGTH": 512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH": 50,  # max length of target text
    "SEED": 42,  # set seed for reproducibility
}

**Perform model fine-tuning**

In [11]:
# T5 accepts prefix of the task to be performed:
# Since we are summarizing, let's add summarize to source text as a prefix
df["Judgment"] = "summarize: " + df["Judgment"]

predictions  = T5Trainer(dataframe=df,
                         source_text="Judgment",
                         target_text="Summary",
                         model_params=model_params,
                         output_dir="outputs"
                        )

[18:18:30] [Model]: Loading t5-base...                                                             ]8;id=709016;file:///tmp/ipykernel_25/2679158037.py\2679158037.py]8;;\:]8;id=864846;file:///tmp/ipykernel_25/2679158037.py#16\16]8;;\
                                                                                                                   

/opt/conda/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5.py:173: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


[18:18:42] [Data]: Reading data...                                                                 ]8;id=918872;file:///tmp/ipykernel_25/2679158037.py\2679158037.py]8;;\:]8;id=498362;file:///tmp/ipykernel_25/2679158037.py#27\27]8;;\
                                                                                                                   

                                                    Sample Data                                                    
+-----------------------------------------------------------------------------------------------------------------+
|                      source_text                       |                       target_text                      |
|--------------------------------------------------------+--------------------------------------------------------|
|          summarize: Appeal No. LXVI of 1949.           | The charge created in respect of municipal property tax|
|Appeal from the High Court of judicature, Bombay, in a  |   by section 212 of the City of Bombay Municipal Act,  |
|  reference under section 66 of the Indian Income tax   | 1888, is an "annual charge not being a capital charge" |
|                      Act, 1022.                        | within the mean ing of section 9 (1) (iv) of the Indian|
|    K.M. Munshi (N. P. Nathvani, with him), for the     |  Income tax Act, 199.2, and the amount of such charge  |
|appellant. ' M.C. Setalvad, Attorney General for India  |  should therefore be deducted in computing the income  |
| (H. J. Umrigar, with him), for the respondent. 1950.   | from such property for the purposes of section 9 of the|
|                        May 26.                         |                 Indian Income tax Act.                 |
| The judgment of the Court was delivered by MEHR CHAND  | The charge in respect of urban immoveable property tax |
|                      MAHAJAN J.                        |  created by the Bombay Finance Act, 1939 is similar in |
|This is an appeal against a judgment of the High Court  | character and the amount of such charge should also be |
|of Judicature at Bombay in an income tax matter and it  |                        deducted.                       |
|raises the question whether munici pal property tax and | The expression "capital charge" in s.9(1) (iv) means a |
|    urban immoveable property tax payable under the     |   charge created for a capital sum,that is to say, a   |
|  relevant Bombay Acts are allowable deductions under   |     charge created to. ' secure the discharge of a     |
|   section 9 (1) (iv) of the Indian Income tax Act.     | liability of a capi tal nature; and an "annual charge" |
|The assessee company is an investment company deriving  |   means a charge to secure an annual liabili ty. 554   |
|   its income from properties in the city of Bombay.    |                                                        |
| For the assessment year 1940 41 the net income of the  |                                                        |
|assessee under the head "property" was computed by the  |                                                        |
|  Income tax Officer in the sum of Rs. 6,21,764 after   |                                                        |
|     deducting from gross rents certain payments.       |                                                        |
|   The company had paid during the relevant year Rs.    |                                                        |
| 1,22,675 as municipal property tax and Rs. 32,760 as   |                                                        |
|                  urban property tax.                   |                                                        |
|   Deduction of these two sums was claimed under the    |                                                        |
|           provisions of section 9 the Act.             |                                                        |
|  Out of the first item a deduction in the sum of Rs.   |                                                        |
|    48,572 was allowed on the ground that this item     |                                                        |
|  represented tenants ' burdens paid by the assessee,   |                                                        |
|          otherwise the claim was disallowed.          

FULL Dataset: (7030, 2)

...

Completed 170

[18:42:07] [Validation Completed.]                                                                ]8;id=906840;file:///tmp/ipykernel_25/2679158037.py\2679158037.py]8;;\:]8;id=788283;file:///tmp/ipykernel_25/2679158037.py#105\105]8;;\
                                                                                                                   

[Model] Model saved @ outputs/model_files

[Validation] Generation on Validation data saved @ outputs/predictions.csv

[Logs] Logs saved @ outputs/logs.txt

**Predictions**

In [12]:
predictions.sample(20)

,Generated Text,Actual Text
1075,"the Governing Body of Guru Charan College, Sil...",R was appointed Professor of Mathematics in a ...
1015,petitioners challenged the constitutional vali...,% These petitions under Article 32 of the Cons...
650,TAX REFERENCE NOS. 6 9 OF 1975 (From the Tax R...,The earliest provision granting exemption of s...
447,the Settlement Officer of Srikakulam suo motu ...,The Settlement Officer under the Madras Estate...
1290,the appellants were carrying on business of cl...,"The business being carried on by a HUF, of whi..."
339,the appellants & respondents Nos. 1 to 3 (in C...,Entry 54 of the Union List I in the Seventh Sc...
914,the petitioners describe themselves as caterin...,"More than a quarter of a century ago, in the S..."
101,the appellant was the wife of Siri Chand Kavir...,Dr. Vimla purchased a car in the name of her m...
70,"the Kerala Agrarina Relations Act, No. IV of 1...",The Kerala Agrarian Relations Act was impugned...
764,"the Sugarcane (Control) Order, 1966, conferred...",The raw material for manufacturing sugar or Kh...
